<a href="https://colab.research.google.com/github/lastinm/ml_hw_notebooks/blob/main/%D0%91%D0%B8%D0%B7%D0%BD%D0%B5%D1%81_%D0%BF%D1%80%D0%B8%D0%BC%D0%B5%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5_%D0%BC%D0%B0%D1%88%D0%B8%D0%BD%D0%BD%D0%BE%D0%B3%D0%BE_%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D1%8F_%D0%98%D1%82%D0%BE%D0%B3%D0%BE%D0%B2%D1%8B%D0%B9_%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Бизнес-применение машинного обучения. Итоговый проект

Маркетинговые агенства хотят проводить успешные рекламные кампании, но рынок сложен, что требует оценивать эффективности разных вариантов, что обуславливает необходимость проведения АБ-тестирования.

Компании заинтересованы в ответах на два вопроса:

1. Будет ли кампания успешной?

2. Если кампания была успешной, насколько этот успех можно объяснить рекламой?

Большинство людей будут видеть рекламу (экспериментальная группа). А небольшая часть людей (контрольная группа) вместо этого увидит объявление государственной службы (PSA) (или ничего) точно такого же размера и в том же месте, где обычно находится реклама.

Идея задания состоит в том, чтобы проанализировать группы, выяснить, была ли реклама успешной, сколько компания может заработать на рекламе и является ли разница между группами статистически значимой.

## ДАТАСЕТ

In [1]:
# подключение диска
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Чтение CSV файла напрямую из zip архива
import pandas as pd
ab_data = pd.read_csv('/content/drive/MyDrive/Data/AB-tests_HW.zip', compression='zip', header=0, sep=',')
ab_data.head()

,Unnamed: 0,user id,test group,converted,total ads,most ads day,most ads hour
0,0,1069124,ad,False,130,Monday,20
1,1,1119715,ad,False,93,Tuesday,22
2,2,1144181,ad,False,21,Tuesday,18
3,3,1435133,ad,False,355,Tuesday,10
4,4,1015700,ad,False,276,Friday,14


## Словарь данных:

**user id**: Идентификатор пользователя (уникальный).

**test group**: Если "ad", то человек видел рекламу, если "psa", то он видел только объявление государственной службы.

**converted**: Если человек купил продукт, то True, иначе False.

**total ads**: Количество рекламы, увиденной человеком.

**most ads day**: День, в который человек увидел наибольшее количество рекламы.

**most ads hour**: Час дня, в который человек увидел наибольшее количество рекламы.

## АНАЛИЗ ДАННЫХ

### Исследование структуры данных

In [4]:
ab_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588101 entries, 0 to 588100
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Unnamed: 0     588101 non-null  int64 
 1   user id        588101 non-null  int64 
 2   test group     588101 non-null  object
 3   converted      588101 non-null  bool  
 4   total ads      588101 non-null  int64 
 5   most ads day   588101 non-null  object
 6   most ads hour  588101 non-null  int64 
dtypes: bool(1), int64(4), object(2)
memory usage: 27.5+ MB


* Имеется 588101 запись и 7 колонок.

* Все колонки содержат ненулевые значения, что говорит о том, что нет пропусков.

* Данные в колонках имеют разные типы. Например, converted — булевый тип, что хорошо. Однако стоит обратить внимание на типы данных других колонок.

### Преобразование столбцов к необходимым типам данных

In [7]:
# Преобразование нужных колонок
ab_data['most ads day'] = ab_data['most ads day'].astype('category')
ab_data['test group'] = ab_data['test group'].astype('category')

In [9]:
ab_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588101 entries, 0 to 588100
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   Unnamed: 0     588101 non-null  int64   
 1   user id        588101 non-null  int64   
 2   test group     588101 non-null  category
 3   converted      588101 non-null  bool    
 4   total ads      588101 non-null  int64   
 5   most ads day   588101 non-null  category
 6   most ads hour  588101 non-null  int64   
dtypes: bool(1), category(2), int64(4)
memory usage: 19.6 MB


In [8]:
ab_data.head()

,Unnamed: 0,user id,test group,converted,total ads,most ads day,most ads hour
0,0,1069124,ad,False,130,Monday,20
1,1,1119715,ad,False,93,Tuesday,22
2,2,1144181,ad,False,21,Tuesday,18
3,3,1435133,ad,False,355,Tuesday,10
4,4,1015700,ad,False,276,Friday,14


### Проверка данных на наличие дубликатов пользователей в обеих группах

Убедимся, что пользователи не попали в обе группы, мы можем сделать следующее:

* Используем метод группировки (groupby) для анализа user id и test group.
* Проверим, сколько уникальных пользователей (user id) прошло через каждую группу.
* Удалим пользователей, которые оказались в обеих группах.

In [11]:
# Поработаем с пользователями
duplicates = ab_data.groupby('user id')['test group'].nunique()
users_in_both_groups = duplicates[duplicates > 1].index

# Отфильтруем дубликаты
ab_data_filtered = ab_data[~ab_data['user id'].isin(users_in_both_groups)]
ab_data_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588101 entries, 0 to 588100
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   Unnamed: 0     588101 non-null  int64   
 1   user id        588101 non-null  int64   
 2   test group     588101 non-null  category
 3   converted      588101 non-null  bool    
 4   total ads      588101 non-null  int64   
 5   most ads day   588101 non-null  category
 6   most ads hour  588101 non-null  int64   
dtypes: bool(1), category(2), int64(4)
memory usage: 19.6 MB


Пользователь состаящие в обоих группах не выявлены.

##  Первичный анализ результатов A/B-тестирования


Рассчитайте вспомогательные показатели для контрольной и тестовой групп:

 * количество посещений сайта:


In [17]:
# Разделение на контрольную и тестовую группы
test_group = ab_data[ab_data['test group'] == 'ad']
control_group = ab_data[ab_data['test group'] == 'psa']

In [19]:
# Подсчет уникальных пользователей
unique_users_test = test_group['user id'].nunique()
unique_users_control = control_group['user id'].nunique()

* суммарное количество совершённых покупок:

In [20]:
# Подсчет суммарного количества совершённых покупок
total_converted_test = test_group['converted'].sum()
total_converted_control = control_group['converted'].sum()

In [21]:
# Вывод результата
print(f"Тестовая группа (ad):")
print(f"Суммарное количество совершённых покупок: {total_converted_test}")
print(f"Количество уникальных пользователей: {unique_users_test}")

print(f"\nКонтрольная группа (psa):")
print(f"Суммарное количество совершённых покупок: {total_converted_control}")
print(f"Количество уникальных пользователей: {unique_users_control}")

Тестовая группа (ad):
Суммарное количество совершённых покупок: 14423
Количество уникальных пользователей: 564577

Контрольная группа (psa):
Суммарное количество совершённых покупок: 420
Количество уникальных пользователей: 23524
